In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [46]:
# Load the training dataset
train_data = pd.read_csv(r'C:\Users\Syed Asad\OneDrive\Desktop\Python Files\Data Analysis\Kaggle Competitions\Titanic Disaster\titanic\train.csv')

# Display the first few rows of the dataset
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
# Check for missing values in the training dataset
missing_values = train_data.isnull().sum()

missing_values

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [48]:
 # Handling missing values

# 1. Fill missing Age values with median
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)

# 2. Convert Cabin column to binary: 1 if cabin info is available, 0 otherwise
train_data['Cabin'] = train_data['Cabin'].notnull().astype(int)

# 3. Fill missing Embarked values with the mode
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)

In [49]:
# Check if there are still any missing values
remaining_missing = train_data.isnull().sum()

remaining_missing

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Feature Engineering

In [50]:
# 1. Extract title from Name
train_data['Title'] = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# Grouping rare titles under a common label "Rare"
rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
train_data['Title'] = train_data['Title'].replace(rare_titles, 'Rare')
train_data['Title'] = train_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')

In [51]:
# 2. Create FamilySize feature
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1


In [52]:
# 3. Create IsAlone feature
train_data['IsAlone'] = 0
train_data.loc[train_data['FamilySize'] == 1, 'IsAlone'] = 1


In [53]:
# 4. One-hot encoding for categorical features
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked', 'Title'])

In [54]:
# Drop columns that we don't need for modeling
train_data.drop(['Name', 'Ticket', 'PassengerId'], axis=1, inplace=True)

In [55]:
# Display the first 5 rows of the processed dataset
train_data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,FamilySize,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,0,3,22.0,1,0,7.2500,0,2,0,0,1,0,0,1,0,0,1,0,0
1,1,1,38.0,1,0,71.2833,1,2,0,1,0,1,0,0,0,0,0,1,0
2,1,3,26.0,0,0,7.9250,0,1,1,1,0,0,0,1,0,1,0,0,0
3,1,1,35.0,1,0,53.1000,1,2,0,1,0,0,0,1,0,0,0,1,0
4,0,3,35.0,0,0,8.0500,0,1,1,0,1,0,0,1,0,0,1,0,0


In [56]:
# Split data into features (X) and target (y)
X = train_data.drop('Survived', axis=1)
y = train_data['Survived']

In [57]:
# Splitting data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
# Hyperparameters to be tuned
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [60]:
# Defining rf_classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

In [61]:
# Grid search
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 216 candidates, totalling 648 fits


C:\Users\Syed Asad\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
324 fits failed out of a total of 648.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
133 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Syed Asad\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Syed Asad\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\Syed Asad\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Syed Asad\anaco

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=2)

In [62]:
# Best parameters from grid search
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score

({'max_depth': None,
  'max_features': 'sqrt',
  'min_samples_leaf': 4,
  'min_samples_split': 10,
  'n_estimators': 50},
 0.8384864494320935)

In [63]:
# Train Random Forest classifier with best parameters obtained above
optimized_rf_classifier = RandomForestClassifier(**best_params, random_state=42)
optimized_rf_classifier.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=4, min_samples_split=10,
                       n_estimators=50, random_state=42)

In [64]:
# Predicting on validation set using optimized model
optimized_y_pred = optimized_rf_classifier.predict(X_val)

In [66]:
# Calculating accuracy for the optimized model
optimized_accuracy = accuracy_score(y_val, optimized_y_pred)
optimized_accuracy

0.8379888268156425

In [67]:
# Preprocess and engineer features for the test dataset
test_data = pd.read_csv(r'C:\Users\Syed Asad\OneDrive\Desktop\Python Files\Data Analysis\Kaggle Competitions\Titanic Disaster\titanic\test.csv')
passenger_ids = test_data['PassengerId']

In [68]:
# Apply the same transformations as training data
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Cabin'] = test_data['Cabin'].notnull().astype(int)
test_data['Embarked'].fillna(test_data['Embarked'].mode()[0], inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)  # Filling missing fares
test_data['Title'] = test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_data['Title'] = test_data['Title'].replace(rare_titles, 'Rare')
test_data['Title'] = test_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data['IsAlone'] = 0
test_data.loc[test_data['FamilySize'] == 1, 'IsAlone'] = 1
test_data = pd.get_dummies(test_data, columns=['Sex', 'Embarked', 'Title'])
test_data.drop(['Name', 'Ticket', 'PassengerId'], axis=1, inplace=True)

In [69]:
# Predicting on the test dataset
test_predictions = optimized_rf_classifier.predict(test_data)

optimized_accuracy, test_predictions[:10]  # Displaying the first 10 predictions for verification

(0.8379888268156425, array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64))

In [71]:
# Preparing the submission dataframe
submission_df = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': test_predictions
})

In [73]:
# Saving the submission to a CSV file
submission_file_path = r'C:\Users\Syed Asad\OneDrive\Desktop\Python Files\Data Analysis\Kaggle Competitions\Titanic Disaster\titanic\titanic_predictions.csv'
submission_df.to_csv(submission_file_path, index=False)

In [2]:
from sklearn.externals import joblib

ImportError: cannot import name 'joblib' from 'sklearn.externals' (C:\Users\Syed Asad\anaconda3\Lib\site-packages\sklearn\externals\__init__.py)